<a href="https://colab.research.google.com/github/AJ-Horch/colab/blob/main/Project_2_guided_completed_SADAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve Any Data Analysis Problem
## Chapter 3 - Project 2: Who are your customers?
### Goals
- explore all 3 datasets
- come up with a common schema to record customers (hat columns of customer data are common across data sources)
- populate new customer dataset with
    - customers who have made a purchase and can be identified in the purchases table, and looking up their deatils in the customer datasets
    - actual customers from the customer database (if they aren;t already referenced in the purchases table)
    - guest checkout from purchase history
- for each "inferred" customer record, store:
    - all relevant customer data
    - source of information
- deduplicate dataset

### Final output
A dataset containing one row per "inferred customer entity"

# Exploring the data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install recordlinkage --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.5 MB/s eta 0:00:00


In [ ]:
import recordlinkage

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving crm_export.csv to crm_export.csv
Saving customer_database.csv to customer_database.csv
Saving purchases.csv to purchases.csv


## Sales

In [ ]:
sales = pd.read_csv('purchases.csv')
print(sales.shape)

(71519, 11)


In [ ]:
sales.head()

,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode
0,2022-10-01 02:26:08+00:00,32701106,2055156924466332447,NaN,shimano,95.21,64c68405-7002-4ce0-9604-a4c2e1f7384b,NaN,MICHAEL,MASON,RG497ZQ
1,2022-10-01 02:28:32+00:00,9400066,2053013566067311601,NaN,jaguar,164.20,3b7d6741-3c82-4c75-8015-6f54b52612e0,7466.0,NaN,NaN,NaN
2,2022-10-01 02:31:01+00:00,1004238,2053013555631882655,electronics.smartphone,apple,1206.40,38c6d3f7-6c32-4fed-bca6-ef98e1746386,NaN,COLE,WILKINSON,SW75TQ
3,2022-10-01 02:33:31+00:00,11300059,2053013555531219353,electronics.telephone,texet,17.48,3398c966-7846-4186-89be-323daad735b9,NaN,MOHAMMED,RICHARDS,RG150RE
4,2022-10-01 02:40:18+00:00,17300751,2053013553853497655,NaN,versace,77.22,11e3a573-01b9-4794-b513-e7d8a4fcac83,31266.0,NaN,NaN,NaN


In [ ]:
# looking for missing values
# this could be achieved by doing .info but easier to see the missing values this way
sales.isnull().sum()

event_time              0
product_id              0
category_id             0
category_code       16739
brand                5707
price                   0
session_id              0
customer_id         18448
guest_first_name    53071
guest_surname       53071
guest_postcode      53071
dtype: int64

over 53k guest names missing. This means that there are 53K registered users and 18,448 guests.

create a column to track guest checkouts.

The thought process here is that it will be easier to distinguish the guest from registered users

In [ ]:
sales["is_guest"] = sales["customer_id"].isnull()
# This line is creating a new col, that takes a boolean value
# based on if there is a customer id or not.
# customer id only exists for registered costumers

Are there cases where quests checkouts also had a customer ID filled in?

In [ ]:
sales[sales["is_guest"] & sales["customer_id"].notnull()]

,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode,is_guest


The code above filters the DataFrame sales to include only those rows where the sale is associated with a guest customer (is_guest is True) and has a non-null customer ID. The result is a DataFrame containing only the relevant rows that meet both conditions.

What about anywhere with neither?

In [ ]:
sales[(sales["is_guest"] == False) & sales["customer_id"].isnull()]

,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode,is_guest


This code does the same as the code above, however with "is_guest" set to False.

Look at the proportion of guest vs non-guest checkouts:


Next on our data quality agenda is checking what percentage of
records are guest checkouts. This is not just for general informational purposes, but also
for us to get a sense of how many customer records we will have to infer. Since guest
checkouts are our weakest signal for a customer record, any guests we add to our
customer database are assumed customers. They are inferred rather than concrete
customer accounts

In [ ]:
print(sales['is_guest'].value_counts())
print(sales['is_guest'].value_counts(normalize=True)) # normalize: If True then the object returned will contain the relative frequencies of the unique values.

False    53071
True     18448
Name: is_guest, dtype: int64
False    0.742055
True     0.257945
Name: is_guest, dtype: float64


Approx 26% are guest checkouts. How many unique customer names does that translate to?

In [ ]:
print("{} rows of purchases with guest checkout".format(len(sales[sales["is_guest"]])))

18448 rows of purchases with guest checkout


Including the postcode with first and surnames makes a unique identity.

In [ ]:
guest_columns = ["guest_first_name", "guest_surname", "guest_postcode"]
unique_guests = sales[guest_columns].drop_duplicates() # et a unique combination of guest columns | the postcode makes this unique
print(f"unique guests = {len(unique_guests)}")
unique_customers = sales['customer_id'].unique()
cust_total = len(unique_customers) + len(unique_guests)
print(f"guests make up : {len(unique_guests) / (cust_total-1)} percentage") # subtract 1 from the unique customer count because NULL is also counted


unique guests = 8301
guests make up : 0.2495640671036017 percentage


This prints a value of 8301 and another just under 0.25, meaning we have 8,301 unique
combinations of guest columns, and once we extract unique customers it turns out a
quarter are indeed not registered and checked out as guests instead. This number won’t
be exact because there could be typos

In [ ]:
# the number of unique registered customer ids
sales["customer_id"].nunique()

24961

Nearly 25k of customers make up the other 75%. There are 53K customer ids out of 71k entries.

Do we have any guest checkouts with a customer ID or a row with neither

In [ ]:
missing_guest_info = (
    sales[(sales['customer_id'].isnull()) &
          ((sales['guest_first_name'].isnull())
           | (sales['guest_surname'].isnull())
           | (sales['guest_postcode'].isnull()))]
)

print(len(missing_guest_info))
missing_guest_info

0


,event_time,product_id,category_id,category_code,brand,price,session_id,customer_id,guest_first_name,guest_surname,guest_postcode,is_guest


### summary: purchases
- we have ~35k unique customers
- plys 18k recordds with guest checkout, which is roughly 8k unique guest customers (not counting dups_

Total expected customer bases approx 30-35k

Customer data available:
- first name
- surname
- postcode


In [ ]:
# export this to its own DataFrame to merge with the other customers later
guest_columns = ["guest_first_name", "guest_surname", "guest_postcode", "is_guest"]
guests = sales.loc[sales["is_guest"], guest_columns].drop_duplicates() # drop dupicates to ensure there is only unique guest data
guests.head()

,guest_first_name,guest_surname,guest_postcode,is_guest
0,MICHAEL,MASON,RG497ZQ,True
2,COLE,WILKINSON,SW75TQ,True
3,MOHAMMED,RICHARDS,RG150RE,True
7,KIAN,MILLS,SW332TF,True
13,RUBY,OWEN,PO377YS,True


In [ ]:
non_guests = (
    pd.DataFrame( # customer ID is a single column so we need to explicitly make it a DataFrame
        sales.loc[sales["customer_id"].notnull(), "customer_id"]
            .unique() # we extract unique customer IDs from non-guest rows
            .astype(int),
        columns=["customer_id"])
)
non_guests.head()

,customer_id
0,7466
1,31266
2,534142828
3,1035
4,6985


Merge the two (accepting that we'll have NULLS)

In [ ]:
sales_customers = pd.concat([non_guests, guests], axis=0, ignore_index=True)
new_col_names = ["customer_id", "first_name", "surname", "postcode", "is_guest"]
sales_customers = sales_customers.set_axis(new_col_names, axis=1) # rename columns

sales_customers["is_guest"] = sales_customers["is_guest"].fillna(False)

# mark the source of the records
sales_customers["in_purchase_data"] = True

Let's santize the name cols

In [ ]:
for col in ["first_name", "surname"]:
    sales_customers[col] = sales_customers[col].str.lower().str.strip()

sales_customers["postcode"] = sales_customers["postcode"].str.strip()

sales_customers

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data
0,7466.0,NaN,NaN,NaN,False,True
1,31266.0,NaN,NaN,NaN,False,True
2,534142828.0,NaN,NaN,NaN,False,True
3,1035.0,NaN,NaN,NaN,False,True
4,6985.0,NaN,NaN,NaN,False,True
...,...,...,...,...,...,...
33256,NaN,poppy,foster,M192EQ,True,True
33257,NaN,sophie,chapman,NW500AS,True,True
33258,NaN,scarlett,shaw,EX86QS,True,True
33259,NaN,michael,harrison,HR280TG,True,True


- in_purchase_data is the source col indicating which dataset the data is in
- is_guest is determined by if there is the value of customer_id. If there's a value then is_guest is True. Nulls in the col are filled with False.
- Will add customer names postcodes later from other datasets

## CRM Data

In [ ]:
crm = pd.read_csv("crm_export.csv")
print(crm.shape)
crm.head()

(7825, 5)


,customer_id,first_name,surname,postcode,age
0,29223,Holly,Rogers,LS475RT,12
1,27826,Daniel,Owen,M902XX,5
2,7432,Eleanor,Russell,HR904ZA,34
3,2569,Paige,Roberts,DE732EP,61
4,9195,Matilda,Young,LS670FU,78


In [ ]:
# check for nulls
crm.isnull().sum()

customer_id    0
first_name     0
surname        0
postcode       0
age            0
dtype: int64

Santize the data so strings are all lowercase and whitespace is removed

In [ ]:
for col in ["first_name", "surname"]:
    crm[col] = crm[col].str.lower().str.strip()

crm["postcode"] = crm["postcode"].str.strip()

crm.head()

,customer_id,first_name,surname,postcode,age
0,29223,holly,rogers,LS475RT,12
1,27826,daniel,owen,M902XX,5
2,7432,eleanor,russell,HR904ZA,34
3,2569,paige,roberts,DE732EP,61
4,9195,matilda,young,LS670FU,78


Verify that customer ID is in fact unique. The usual trick
for this is to group by the customer ID and find instances where there are multiple rows
in a group. If customer IDs are unique, no records should be returned.


In [ ]:
crm.groupby("customer_id").size().loc[lambda x: x > 1]
# how is this working?

Series([], dtype: int64)

Here we use groupby and size to count up how many records we have per customer ID
and use loc to filter instances where there is more than one. The Python output is
Series([], dtype: int64) which indicates no records were found, as the empty
square brackets represent an empty collection in Python. This means customer IDs are
indeed unique.

No, what about duplicate information??

In [ ]:
print(len(crm))
print(len(crm.drop(columns="customer_id").drop_duplicates()))

7825
7419


In [ ]:
print(f"rows = {len(crm)}")
print(f"unique combinations of customer informaiton = {len(crm.drop(columns='customer_id').drop_duplicates())}")

rows = 7825
unique combinations of customer informaiton = 7419


Now join CRM dat to purchases where purchases have a customer ID not guests

In [ ]:
print(len(sales_customers))
sales_customers.isnull().sum()

33261


customer_id          8300
first_name          24961
surname             24961
postcode            24961
is_guest                0
in_purchase_data        0
dtype: int64

In [ ]:
sales_and_crm_customers = sales_customers.merge(crm, on="customer_id", how="left")
print(len(sales_and_crm_customers))
sales_and_crm_customers.isnull().sum()

33261


customer_id          8300
first_name_x        24961
surname_x           24961
postcode_x          24961
is_guest                0
in_purchase_data        0
first_name_y        26147
surname_y           26147
postcode_y          26147
age                 26147
dtype: int64

ne pandas-specific peculiarity is that columns that appear in both tables get a _x and
_y suffix. The ones with _x are from the source data, the purchases, and the _y suffix is
given to the merged data, in this case, the CRM data. Given that there were around
33,000 rows in the sales data and 26,000 rows missing from the newly-added CRM
customer columns, we can see we matched around 7,000 rows on customer ID. That is,
customers in 7,000 purchases had their records stored in the CRM table. What we have
now is a dataset where 7,000 customer records are in columns ending in _y, which we
should merge into the ones marked _x, which contain customer data from guest
checkouts. First, we define a filter to select only rows with a customer ID, thus excluding
guests, and rows with customer information in the _y suffixed columns.

Update record source for merged records

In [ ]:
# now we have dupliate customer details columns, so merge them
# into the old ones
merged_customers_filter = (
    (sales_and_crm_customers["customer_id"].notnull()) # only for actual customers
    # only if they have at least first or surname filled in
    # meaning we've found a matchin our CRM data
    & ((sales_and_crm_customers["first_name_y"].notnull())
      | (sales_and_crm_customers["surname_y"].notnull()))
)

sales_and_crm_customers.loc[merged_customers_filter, "in_crm_data"] = True
sales_and_crm_customers.loc[~merged_customers_filter, "in_crm_data"] = False

sales_and_crm_customers["in_crm_data"].value_counts()
# that does the ~ symbol do???

False    26147
True      7114
Name: in_crm_data, dtype: int64

Now merge data into single versions of customer information

Here we simply copied over the first name, surname, and postcode to overwrite the
missing values in _x suffixed columns with CRM customer data in the _y suffixed ones.
Now we’re ready to remove the latter.

In [ ]:
sales_and_crm_customers.loc[merged_customers_filter, ["first_name_x", "surname_x", "postcode_x"]] = (
sales_and_crm_customers.loc[merged_customers_filter, ["first_name_y", "surname_y", "postcode_y"]]
    .values
)

# Drop duplcate columns and rename
sales_and_crm_customers = (
    sales_and_crm_customers
    .drop(columns=["first_name_y", "surname_y", "postcode_y"])
    .rename(columns={
        "first_name_x": "first_name",
        "surname_x": "surname",
        "postcode_x": "postcode"
    })
)

print(sales_and_crm_customers.isnull().sum())
sales_and_crm_customers.head()

customer_id          8300
first_name          17847
surname             17847
postcode            17847
is_guest                0
in_purchase_data        0
age                 26147
in_crm_data             0
dtype: int64


,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data
0,7466.0,NaN,NaN,NaN,False,True,NaN,False
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False
3,1035.0,NaN,NaN,NaN,False,True,NaN,False
4,6985.0,NaN,NaN,NaN,False,True,NaN,False


We now need to add CRM customers who are not in our purchase history

What remains is to check for, and add, customer details that exist in our
CRM system but do not appear in our purchases. There may be reasons for this; perhaps
those customers bought something on the phone and those sales do not get recorded in
the same place. Whatever the reason, it is a possibility we need to account for to ensure
full coverage.

In [ ]:
crm_ids_to_add = list(set(crm["customer_id"].unique()) - set(sales_and_crm_customers["customer_id"].unique()))
print(len(crm_ids_to_add))

711


Here we employ a Python trick to subtract one set of customer IDs from another, leaving
us with the difference. Here, “set” refers to the rigorous mathematical definition of a
unique collection of items, and “difference” means the subtraction of one set from
another, leaving us with only customer IDs who appear in the CRM data but not in
purchases. The output tells us there are 711 such customers, whose details need to be
added to our growing customer dataset. We simply concatenate/union the data with the
customers corresponding to the IDs we have just selected.

In [ ]:
print(len(sales_and_crm_customers))

sales_and_crm_customers = (
    pd.concat([sales_and_crm_customers, crm[crm["customer_id"].isin(crm_ids_to_add)]],
             axis=0,ignore_index=True)
)

print(len(sales_and_crm_customers))

33261
33972


In [ ]:
sales_and_crm_customers.isnull().sum()

customer_id          8300
first_name          17847
surname             17847
postcode            17847
is_guest              711
in_purchase_data      711
age                 26147
in_crm_data           711
dtype: int64

Update missing record sources accordingly

In [ ]:
sales_and_crm_customers["is_guest"] = sales_and_crm_customers["is_guest"].fillna(False)
sales_and_crm_customers["in_purchase_data"] = sales_and_crm_customers["in_purchase_data"].fillna(False)
sales_and_crm_customers["in_crm_data"] = sales_and_crm_customers["in_crm_data"].fillna(True)

sales_and_crm_customers.isnull().sum()

customer_id          8300
first_name          17847
surname             17847
postcode            17847
is_guest                0
in_purchase_data        0
age                 26147
in_crm_data             0
dtype: int64

In [ ]:
print(len(sales_and_crm_customers))
sales_and_crm_customers.head()

33972


,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data
0,7466.0,NaN,NaN,NaN,False,True,NaN,False
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False
3,1035.0,NaN,NaN,NaN,False,True,NaN,False
4,6985.0,NaN,NaN,NaN,False,True,NaN,False


This is not one row per customer form BOTH the CRM data and purchase history including:
- guest checkouts from the purchase table
- customers in the purchases table who had a customerID now have thier data included (if they were in the CRM data)
- customers who are in the CRM database but have made no purchases


## Customers

In [ ]:
customers = pd.read_csv('customer_database.csv')
print(customers.shape)
customers.head()

(23476, 5)


,customer_id,first_name,surname,postcode,age
0,1641,Rhys,Richards,DE456EZ,45
1,24796,Maisie,Young,SW433XX,16
2,14358,Nathan,King,NW49TU,58
3,15306,Jack,Moore,NW908RR,26
4,24971,Alexander,Roberts,SW500HW,85


In [ ]:
# check for nulls
customers.isnull().sum()

customer_id    0
first_name     0
surname        0
postcode       0
age            0
dtype: int64

Similar santizing to CRM

In [ ]:
for col in ['first_name', 'surname']:
    customers[col] = customers[col].str.lower().str.strip()

customers['postcode'] = customers['postcode'].str.strip()

In [ ]:
customers.groupby("customer_id").size().loc[lambda x: x>1]

Series([], dtype: int64)

In [ ]:
print(f" rows = {len(customers)}")
unique_customers = customers.drop(columns="customer_id").drop_duplicates()
print(f"unique combinations of customers = {(len(unique_customers))}")

 rows = 23476
unique combinations of customers = 19889


Join this data to our growing merged customer data

In [ ]:
print(len(sales_and_crm_customers))

all_customers = sales_and_crm_customers.merge(customers, on="customer_id", how="left")
print(len(all_customers))
all_customers.head()

33972
33972


,customer_id,first_name_x,surname_x,postcode_x,is_guest,in_purchase_data,age_x,in_crm_data,first_name_y,surname_y,postcode_y,age_y
0,7466.0,NaN,NaN,NaN,False,True,NaN,False,eve,richards,HR90PT,45.0
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True,NaN,NaN,NaN,NaN
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False,NaN,NaN,NaN,NaN
3,1035.0,NaN,NaN,NaN,False,True,NaN,False,luca,gibson,DE256NH,30.0
4,6985.0,NaN,NaN,NaN,False,True,NaN,False,mia,rogers,HR662RP,43.0


In [ ]:
# now we have duplicate customer detail columns, so merge them into the old ones
merged_customers_filter = (
    (all_customers["customer_id"].notnull()) # only for actual customers
    # only if they have at least first or surname filled in
    & ((all_customers["first_name_y"].notnull())
       | (all_customers["surname_y"].notnull()))
)

all_customers.loc[merged_customers_filter, "in_customer_data"] = True
all_customers.loc[~merged_customers_filter, "in_customer_data"] = False

all_customers["in_customer_data"].value_counts()

True     22053
False    11919
Name: in_customer_data, dtype: int64

In [ ]:
all_customers.isnull().sum()

customer_id          8300
first_name_x        17847
surname_x           17847
postcode_x          17847
is_guest                0
in_purchase_data        0
age_x               26147
in_crm_data             0
first_name_y        11919
surname_y           11919
postcode_y          11919
age_y               11919
in_customer_data        0
dtype: int64

Fill in customer information if newly added

In [ ]:
update_filter = (
    (all_customers["in_customer_data"])
    & (all_customers["first_name_x"].isnull())
    & (all_customers["surname_x"].isnull())
)
print(len(all_customers))

all_customers.loc[update_filter, ["first_name_x", "surname_x", "postcode_x", "age_x"]] = (
    all_customers.loc[update_filter, ["first_name_y", "surname_y", "postcode_y", "age_y"]].values
)

all_customers = (
    all_customers
    .drop(columns=["first_name_y", "surname_y", "postcode_y", "age_y"])
    .rename(columns={
        "first_name_x": "first_name",
        "surname_x": "surname",
        "age_x": "age",
        "postcode_x": "postcode"
    })
)

print(len(all_customers))
all_customers.isnull().sum()

33972
33972


customer_id         8300
first_name          1248
surname             1248
postcode            1248
is_guest               0
in_purchase_data       0
age                 9548
in_crm_data            0
in_customer_data       0
dtype: int64

Add customer from customer DB but not in main data

In [ ]:
customer_ids_to_add = list(set(customers["customer_id"].unique()) - set(all_customers["customer_id"].unique()))
print(len(customer_ids_to_add))

1423


In [ ]:
print(len(all_customers))

all_customers = (
    pd.concat([all_customers, customers[customers["customer_id"].isin(customer_ids_to_add)]],
             axis=0, ignore_index=True)
)
print(len(all_customers))

33972
35395


2134 customers with no purchase

In [ ]:
all_customers.isnull().sum()

customer_id         8300
first_name          1248
surname             1248
postcode            1248
is_guest            1423
in_purchase_data    1423
age                 9548
in_crm_data         1423
in_customer_data    1423
dtype: int64

Ensure source data is correct

At this point if we have missing data fro whether someone was a guest, it means they weren't (sne we can only mark guests using the purchse data and we've alread done that)

In [ ]:
all_customers["is_guest"] = all_customers["is_guest"].fillna(False)
all_customers["in_purchase_data"] = all_customers["in_purchase_data"].fillna(False)
all_customers["in_crm_data"] = all_customers["in_crm_data"].fillna(False)
all_customers["in_customer_data"] = all_customers["in_customer_data"].fillna(False)

all_customers.isnull().sum()

customer_id         8300
first_name          1248
surname             1248
postcode            1248
is_guest               0
in_purchase_data       0
age                 9548
in_crm_data            0
in_customer_data       0
dtype: int64

In [ ]:
all_customers.head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data
0,7466.0,eve,richards,HR90PT,False,True,45.0,False,True
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True,False
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False,False
3,1035.0,luca,gibson,DE256NH,False,True,30.0,False,True
4,6985.0,mia,rogers,HR662RP,False,True,43.0,False,True


In [ ]:
len(all_customers)

35395

35,395 customers, 23,713 are registered guests, 8300 are guests and 1248 are unidientified.

## Deduplicate

Easy one first, remve all excat duplicates (which might have happened when exported data from purchases where one row is a purchase, not a customer)

In [ ]:
print(len(all_customers))
all_customers = all_customers.drop_duplicates()
print(len(all_customers))

35395
35395


Now we have all possible customer records in one place including:

- those purchased with a customer ID (whether or not that ID is in another database)
- those purchased as guests
- those in the CRM database who haven't made a purchase
- those in the customer database who haven't made a purchase

Let's count up all these eventualities:

In [ ]:
identified_customers = (
    all_customers[(all_customers["customer_id"].notnull())
                 & (all_customers["in_purchase_data"])
                 & ((all_customers["in_crm_data"])
                 | (all_customers["in_customer_data"]))]
)

guest = all_customers[all_customers["is_guest"]]

customer_ids_not_found = (
    all_customers[(all_customers["customer_id"].notnull())
                & (all_customers["first_name"].isnull())
                & (all_customers["surname"].isnull())]
)

customer_data_only = (
    all_customers[((all_customers["in_crm_data"].notnull())
                 | (all_customers["in_customer_data"])
                  )
                  & (all_customers["in_purchase_data"] == False)]
)

print(len(all_customers), len(identified_customers))
print(len(guests), len(customer_ids_not_found), len(customer_data_only))

35395 23713
8300 1248 2134


In [ ]:
print(f"Size of customer database (not yet deduplicated): {len(all_customers)}")
print(f"Identified customers (with an ID, present in a lookup database): {len(identified_customers)}")
print(f"Guest checkouts: {len(guests)}")
print(f"Unidentified customer IDs in purchases: {len(customer_ids_not_found)}")
print(f"Customers with no purchases present in CRM or customer data: {len(customer_data_only)}")

Size of customer database (not yet deduplicated): 35395
Identified customers (with an ID, present in a lookup database): 23713
Guest checkouts: 8300
Unidentified customer IDs in purchases: 1248
Customers with no purchases present in CRM or customer data: 2134


In [ ]:
assert len(all_customers) == len(identified_customers) + len(guests) + len(customer_ids_not_found) + len(customer_data_only)

Roughly lloking at 35,000 customers. Now need to deal with two types of duplication:
- differnt customer IDs encoding exactly the same info (excat dups)
- differnt customer IDs encoding almost excatly the same info (fuzzy dups)

Identify dups by consolidsting them into a main record whihc will be the best guess at a customer database

How to identify a dup
- different customer ID (or NO customer ID in the case of guests)
- but the same details
    - first name, surname, postcode, age (no in guest accounts)

Want to have a "main" recod where I can also recordd all the IDs that we think refer to same account

Guests dont have IDs - give them some

In [ ]:
all_customers["customer_id"].agg(["min", "max"])

min            1.0
max    566239774.0
Name: customer_id, dtype: float64

The range of existing IDs is quuite lage, maybe we're safer with nearive numbers for guests. Culd also go to a much higher range but somme of the IDs are already 9 digits. Could also extend into 12 digit numbers or maybe even alphbetical but using negative numbers works fine.

In [ ]:
all_guests = all_customers[all_customers["is_guest"]].copy()

# go from -1 to -N as needed
new_ids = np.arange(-1, -(len(all_guests) + 1), -1)

all_customers.loc[all_customers["is_guest"], "customer_id"] = new_ids
all_customers.head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data
0,7466.0,eve,richards,HR90PT,False,True,45.0,False,True
1,31266.0,harley,palmer,HR250EJ,False,True,33.0,True,False
2,534142828.0,NaN,NaN,NaN,False,True,NaN,False,False
3,1035.0,luca,gibson,DE256NH,False,True,30.0,False,True
4,6985.0,mia,rogers,HR662RP,False,True,43.0,False,True


In [ ]:
all_customers[all_customers["is_guest"]].head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data
24961,-1.0,michael,mason,RG497ZQ,True,True,NaN,False,False
24962,-2.0,cole,wilkinson,SW75TQ,True,True,NaN,False,False
24963,-3.0,mohammed,richards,RG150RE,True,True,NaN,False,False
24964,-4.0,kian,mills,SW332TF,True,True,NaN,False,False
24965,-5.0,ruby,owen,PO377YS,True,True,NaN,False,False


Convert customer ID to an integer to make it easier to use

In [ ]:
all_customers["customer_id"] = all_customers["customer_id"].astype(int)

Now we try to deduplicate everything including guest accounts

First, we create an object, duplicates, which is a list of all customer records which
are identical to one another in the columns we specified. The keep=False parameter
ensures we keep all relevant records, not just the duplicate ones. Having the keep
parameter as anything else would actually drop the first instance, and only keep the
other rows, the duplicates.

In [ ]:
columns_to_consider = ["first_name", "surname", "postcode"]

duplicates = all_customers[all_customers.duplicated(subset=columns_to_consider, keep=False)]

# Create a dictionary mapping duplicates to customer IDs
# create a lookup dictionary where each customer ID is linked to all the other
# records which are its duplicates. A sample of this dictionary is shown in figure 3.21.
duplicate_dict = duplicates.groupby(columns_to_consider)["customer_id"].apply(list).to_dict()

# Add a new col for the duplicated customer ids
all_customers["other_customer_ids"] = all_customers.apply(lambda x: duplicate_dict.get((x["first_name"], x["surname"], x["postcode"])), axis=1)

all_customers

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids
0,7466,eve,richards,HR90PT,False,True,45.0,False,True,None
1,31266,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]"
2,534142828,NaN,NaN,NaN,False,True,NaN,False,False,None
3,1035,luca,gibson,DE256NH,False,True,30.0,False,True,None
4,6985,mia,rogers,HR662RP,False,True,43.0,False,True,None
...,...,...,...,...,...,...,...,...,...,...
35390,14295,erin,morgan,NW481EN,False,False,63.0,False,False,"[-8152, 14295]"
35391,28025,aaron,harris,SO265RP,False,False,66.0,False,False,None
35392,4220,grace,mitchell,EX709AR,False,False,20.0,False,False,None
35393,13086,oliver,hall,NW277BU,False,False,58.0,False,False,None


Harley Palmer at postcode HR250EJ has
two customer records, IDs 31266 and 5411. Strictly speaking, our other_customer_ids
column should not be self-referential, so we should remove a customer’s own ID from it.
We can create a small function to do that and apply it to the rows with duplicates.

There are records with multiple linked accounts:

In [ ]:
all_customers[all_customers["other_customer_ids"].notnull()].head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids
1,31266,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]"
5,26434,bailey,richardson,SW988AF,False,True,31.0,True,True,"[26434, 27761]"
6,28961,skye,johnson,M80NA,False,True,54.0,False,True,"[28961, 12140]"
10,12586,max,moore,M902XX,False,True,20.0,True,False,"[12586, -6174, 32914]"
12,22825,alicia,wood,SO879UN,False,True,24.0,False,True,"[22825, 28495]"


What I eant now is for each duplicate combo, mark one record as the "main" so that wehn I count the "main" records I get a sence of how many unique values there are.

First I needd to remove each record from its own duplicate list

In [ ]:
def remove_own_record(row):
    ids = list(row["other_customer_ids"])
    ids.remove(row["customer_id"])
    return ids

all_customers.loc[all_customers["other_customer_ids"].notnull(), "duplicate_customer_ids"] = (
    all_customers[all_customers["other_customer_ids"].notnull()].apply(remove_own_record, axis=1)
)

all_customers[all_customers["other_customer_ids"].notnull()].head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids
1,31266,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]",[5411]
5,26434,bailey,richardson,SW988AF,False,True,31.0,True,True,"[26434, 27761]",[27761]
6,28961,skye,johnson,M80NA,False,True,54.0,False,True,"[28961, 12140]",[12140]
10,12586,max,moore,M902XX,False,True,20.0,True,False,"[12586, -6174, 32914]","[-6174, 32914]"
12,22825,alicia,wood,SO879UN,False,True,24.0,False,True,"[22825, 28495]",[28495]


In [ ]:
all_customers[all_customers["customer_id"].isin([31266, 5411])]

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids
1,31266,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]",[5411]
7208,5411,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]",[31266]


We'll add a 'rank" to eachrow based on when I encounter a combination of name + postcode

Anything with rank 1 becomes the "main" record

Whichever representation you choose, you still need to decide which customer record
is the main one. One method is to simply use the first one you encounter. It is unlikely to
make a big difference, but a more principled way would be to use a better metric, like
number of transactions, total spend, etc. to decide which customer record deserves
“main” status.
The technical “trick” to create the flag is to create a column that gives each duplicate
a rank, a row number in the order they are encountered. Anything with a rank of 1
simply becomes a main account. This approach will work for duplicates and unique
records, as the first instance of a combination of customer details will always have a
rank of 1.


In [ ]:
# first and a rank er firstname-surname-postcodec combination
all_customers["rank"] = all_customers.groupby(columns_to_consider).cumcount()+1

all_customers[all_customers["customer_id"].isin([31266, 5411])]

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids,rank
1,31266,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]",[5411],1.0
7208,5411,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]",[31266],2.0


In [ ]:
all_customers.loc[all_customers["rank"] == 1, "is_main"] = True
all_customers["is_main"] = all_customers["is_main"].fillna(False)

all_customers = all_customers.drop(columns="rank")

print(f"Total customers in DB: {len(all_customers)}")
print(f"Of which {len(all_customers[all_customers['is_main']])} are unique/main records")

Total customers in DB: 35395
Of which 27394 are unique/main records


This assumes
that two customers with the same name and postcode are the same customer and that
there are only exact duplicates. To ensure our solution is as accurate as possible given
the data, we can try to match records that are almost identical.

Assumptions/limitations:

- currently the first instance has become the "main" record, this could be improved to choose the one with the highest amount of spend for example (assuming there are linked purchases)
- we assumed all data has to match exactly, so there could be duplicates e.g. with misspellings/typos

## Record linkage

In [ ]:
# index the dataframes
indexer = recordlinkage.Index() # create an Index Object
indexer.block('postcode') # Mark postcode as a column to use for indexing
candidate_links = indexer.index(all_customers.set_index("customer_id")) # Apply the indexing to the data

# set up the comparison rules
compare = recordlinkage.Compare() # Create a comparison object
# Names should be fuzzy comparisons, anything over 85% similar is a match
compare.string('first_name', 'first_name', method='damerau_levenshtein', threshold=0.85, label="first_name")
compare.string('surname', 'surname', method='damerau_levenshtein', threshold=0.85, label="surname")
# Postcodes should match exactly.
compare.exact('postcode', 'postcode', label="postcode")

# create the comparison vectors
compare_vectors = compare.compute(candidate_links, all_customers.set_index("customer_id"))

Here we use the Damerau Levenshtein method, which is a
measure of edit distance – that is, the number of individual character edits required to
get from one string to another. The higher the distance, the less similar the two strings
are. Here you could experiment with different comparison methods and observe the
results.

In [ ]:
compare_vectors

first_name  surname  postcode
customer_id_1 customer_id_2                               
7523           7466                 0.0      0.0         1
7492           7466                 0.0      0.0         1
               7523                 0.0      0.0         1
7518           7466                 0.0      0.0         1
               7523                 0.0      0.0         1
...                                 ...      ...       ...
17659         -7621                 1.0      1.0         1
              -7924                 0.0      0.0         1
               17654                0.0      0.0         1
               17680                0.0      0.0         1
               27928                0.0      0.0         1

[1020864 rows x 3 columns]

In [ ]:
matches = compare_vectors[compare_vectors.sum(axis=1) == 3]
matches

,,first_name,surname,postcode
customer_id_1,customer_id_2,,,
7469,28769,1.0,1.0,1
28583,7468,1.0,1.0,1
32670,7485,1.0,1.0,1
31690,7476,1.0,1.0,1
32600,7516,1.0,1.0,1
...,...,...,...,...
17677,32903,1.0,1.0,1
-489,30716,1.0,1.0,1
-2407,17682,1.0,1.0,1


Merge these customer IDs back to the original dataset and see if we’ve improved
our record linking attempts from before.

In [ ]:
# Create a new DataFrame containg only the two columns of customer id
match_df = pd.DataFrame (
    data=matches.index.tolist(),
    columns=["customer_id_1", "customer_id_2"]
)
# join customer data on the first customer ID
matched = all_customers.merge(match_df,
                              left_on="customer_id", right_on="customer_id_1", how='left')
# join customer data on the seccond customer ID
matched = matched.merge(match_df,
                              left_on="customer_id", right_on="customer_id_2", how='left')

matched

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids,is_main,customer_id_1_x,customer_id_2_x,customer_id_1_y,customer_id_2_y
0,7466,eve,richards,HR90PT,False,True,45.0,False,True,None,NaN,True,NaN,NaN,NaN,NaN
1,31266,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]",[5411],True,NaN,NaN,5411.0,31266.0
2,534142828,NaN,NaN,NaN,False,True,NaN,False,False,None,NaN,False,NaN,NaN,NaN,NaN
3,1035,luca,gibson,DE256NH,False,True,30.0,False,True,None,NaN,True,NaN,NaN,NaN,NaN
4,6985,mia,rogers,HR662RP,False,True,43.0,False,True,None,NaN,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35804,14295,erin,morgan,NW481EN,False,False,63.0,False,False,"[-8152, 14295]",[-8152],False,14295.0,-8152.0,NaN,NaN
35805,28025,aaron,harris,SO265RP,False,False,66.0,False,False,None,NaN,True,NaN,NaN,NaN,NaN
35806,4220,grace,mitchell,EX709AR,False,False,20.0,False,False,None,NaN,True,4220.0,30722.0,NaN,NaN
35807,13086,oliver,hall,NW277BU,False,False,58.0,False,False,None,NaN,True,NaN,NaN,NaN,NaN


In [ ]:
matched[matched["customer_id"] == 30730]

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids,is_main,customer_id_1_x,customer_id_2_x,customer_id_1_y,customer_id_2_y
4829,30730,harvey,robertson,NW436BL,False,True,34.0,False,True,"[30730, 28816, 13817, 13814]","[13814, 13817, 28816]",True,NaN,NaN,28816.0,30730.0
4830,30730,harvey,robertson,NW436BL,False,True,34.0,False,True,"[30730, 28816, 13817, 13814]","[13814, 13817, 28816]",True,NaN,NaN,13817.0,30730.0
4831,30730,harvey,robertson,NW436BL,False,True,34.0,False,True,"[30730, 28816, 13817, 13814]","[13814, 13817, 28816]",True,NaN,NaN,13814.0,30730.0


In [ ]:
# Write a function t collet all linked customer IDs for a given customer ID
def merge_duplicates(group):
  duplicate_list = []
  if np.isnan(group["customer_id_1_y"].values[0]) == False:
              duplicate_list.extend(group["customer_id_1_y"].tolist())
  if np.isnan(group["customer_id_2_x"].values[0]) == False:
              duplicate_list.extend(group["customer_id_2_x"].tolist())
  if len (duplicate_list) > 0:
              return sorted(list(set([int(x) for x in duplicate_list])))
  return np.nan



In [ ]:
matched.head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids,is_main,customer_id_1_x,customer_id_2_x,customer_id_1_y,customer_id_2_y
0,7466,eve,richards,HR90PT,False,True,45.0,False,True,None,NaN,True,NaN,NaN,NaN,NaN
1,31266,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]",[5411],True,NaN,NaN,5411.0,31266.0
2,534142828,NaN,NaN,NaN,False,True,NaN,False,False,None,NaN,False,NaN,NaN,NaN,NaN
3,1035,luca,gibson,DE256NH,False,True,30.0,False,True,None,NaN,True,NaN,NaN,NaN,NaN
4,6985,mia,rogers,HR662RP,False,True,43.0,False,True,None,NaN,True,NaN,NaN,NaN,NaN


In [ ]:
linkages = (
    matched.groupby("customer_id")
    .apply(merge_duplicates).reset_index(name="linked_duplicates")
)

In [ ]:
all_customers = all_customers.merge(linkages, on="customer_id", how="left")
all_customers.head()

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids,is_main,linked_duplicates
0,7466,eve,richards,HR90PT,False,True,45.0,False,True,None,NaN,True,NaN
1,31266,harley,palmer,HR250EJ,False,True,33.0,True,False,"[31266, 5411]",[5411],True,[5411]
2,534142828,NaN,NaN,NaN,False,True,NaN,False,False,None,NaN,False,NaN
3,1035,luca,gibson,DE256NH,False,True,30.0,False,True,None,NaN,True,NaN
4,6985,mia,rogers,HR662RP,False,True,43.0,False,True,None,NaN,True,NaN


In [ ]:
all_customers["duplicate_customer_ids"] = all_customers.loc[all_customers["duplicate_customer_ids"].notnull(), "duplicate_customer_ids"].apply(lambda x: sorted(x))


In [ ]:
(
    all_customers[(all_customers["duplicate_customer_ids"].notnull())
      & (all_customers["linked_duplicates"].notnull())
      & (all_customers["duplicate_customer_ids"] != all_customers["linked_duplicates"])]
)

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids,is_main,linked_duplicates
2197,10383,scarlett,jackson,M284ZB,False,True,35.0,True,True,"[10383, 10386]",[10386],True,"[10386, 27536, 28786]"
7203,19549,summer,anderson,RG546PY,False,True,47.0,True,False,"[19549, 19551]",[19551],True,"[19551, 28832]"
9445,16969,georgia,scott,PO466DY,False,True,9.0,False,True,"[16969, 16966]",[16966],True,"[16966, 32299]"
13277,5390,josh,simpson,HR235FS,False,True,81.0,False,True,"[5390, 32354]",[32354],True,"[32354, 33132]"
14692,32354,josh,simpson,HR235FS,False,True,81.0,False,True,"[5390, 32354]",[5390],False,"[5390, 33132]"
16230,19551,summer,anderson,RG546PY,False,True,38.0,False,True,"[19549, 19551]",[19549],False,"[19549, 28832]"
21022,16966,georgia,scott,PO466DY,False,True,44.0,False,True,"[16969, 16966]",[16969],False,"[16969, 32299]"
24809,10386,scarlett,jackson,M284ZB,False,True,12.0,False,True,"[10383, 10386]",[10383],False,"[10383, 27536, 28786]"
28568,-3608,isabella,harrison,NW908RT,True,True,NaN,False,False,"[-3608, 15377]",[15377],True,"[15376, 15377]"
31139,-6179,lily,chapman,LS238QF,True,True,NaN,False,False,"[-6179, 7936, 7934]","[7934, 7936]",True,"[7934, 7936, 32787]"


In [ ]:
all_customers[all_customers["customer_id"].isin([10383, 10386, 27536])]

,customer_id,first_name,surname,postcode,is_guest,in_purchase_data,age,in_crm_data,in_customer_data,other_customer_ids,duplicate_customer_ids,is_main,linked_duplicates
2197,10383,scarlett,jackson,M284ZB,False,True,35.0,True,True,"[10383, 10386]",[10386],True,"[10386, 27536, 28786]"
10596,27536,scariett,jackson,M284ZB,False,True,12.0,False,True,None,NaN,True,"[10383, 10386]"
24809,10386,scarlett,jackson,M284ZB,False,True,12.0,False,True,"[10383, 10386]",[10383],False,"[10383, 27536, 28786]"


In [ ]:
print(len(all_customers[all_customers["is_main"] == False]))
print(len(all_customers[all_customers["linked_duplicates"].notnull()]))

8001
13689
